In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from noise_utils import *
from torchvision import datasets, transforms as T
from adversarial import *
import pandas as pd

is_cuda_available = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda_available else 'cpu')

test_transform = T.Compose ([
    T.Resize(255),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_data = datasets.ImageFolder("./data/flowers/test", transform=test_transform)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32)

def attack_network(name, type): 
    storage = {}
    
    if name == "alexnet":
        estimator = models.alexnet()
    elif name == "resnet18":
        estimator = models.resnet18()
    else:
        name = "squeezenet1_0"
        estimator = models.squeezenet1_0()
        
    name = "./trained_models/" + name
    states = ["none", "gaussian", "speckle", "s&p", "eraser", "fgsm", "pgd"]
        
    for state in states:
        saved_model = name + "_" + state + ".pth"
        estimator.load_state_dict(torch.load(saved_model))
        estimator = estimator.to(device)
        criterion = nn.CrossEntropyLoss()
            
        if type == "fgsm":
            adversary = FGSM(estimator, criterion, device)
        else:
            adversary = PGD(estimator, criterion, device)
        
        acc = adversary.attack(testloader)
        storage[saved_model[:-4]] = acc
        
    return storage

In [ ]:
storage = attack_network("alexnet", "pgd")
alexnet_df = pd.DataFrame(storage.items(), columns=["Model", "Accuracy"])
alexnet_df

In [ ]:
storage = attack_network("resnet18", "fgsm")
resnet18_df = pd.DataFrame(storage.items(), columns=["Model", "Accuracy"])
resnet18_df

In [ ]:
storage = attack_network("squeezenet1_0", "fgsm")
squeezenet1_0_df = pd.DataFrame(storage.items(), columns=["Model", "Accuracy"])
squeezenet1_0_df